```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 问题

#h(2em) 美国某三种股票（A，B，C）12 年（1943—1954 年）的价格（已经包括了分红在内）每年的增长情况如表 9.4 所示（表中还给出了相应年份的500种股票的价格指数的增长情况）。例如，表中第一个数据 1.300 的含义是股票 A 在 1943 年的年末价值是其年初价值的 1.300 倍，即收益为 30%，其余数据的含义依此类推。假设你在1955年时有一笔资金准备投资这三种股票，并期望年收益率至少达到 15%，那么你应当如何投资？此外，考虑以下问题：

#h(2em) (1) 期望的年收益率在 10%～100% 变化时，投资组合和相应的风险如何变化？

#h(2em) (2) 假设除了上述三种股票外，投资人还有一种无风险的投资方式，如购买国库券。假设国库券的年收益率为 5%，如何考虑该投资问题？

#h(2em) (3) 假设你手上目前握有的股票比例为：股票 A 占 50%，B 占 35%，C 占 15%。这个比例与你得到的最优解可能有所不同，但实际股票市场上每次股票买卖通常总有交易费，例如按交易额的 1% 收取交易费，这时你是否仍需要对手上的股票进行买卖（换手），以便满足“最优解”的要求？

#table(
  columns: (1fr, 1fr, 1fr, 1fr, 1fr),
  stroke: none,
  align: center,
  table.hline(),
  table.header(
    [年份],
    [股票 A],
    [股票 B],
    [股票 C],
    [股票指数],
  ),
  table.hline(),
  [1943],
  [1.300],
  [1.225],
  [1.149],
  [1.258997],
  [1944],
  [1.103],
  [1.290],
  [1.260],
  [1.197526],
  [1945],
  [1.216],
  [1.216],
  [1.419],
  [1.364361],
  [1946],
  [0.954],
  [0.728],
  [0.922],
  [0.919287],
  [1947],
  [0.929],
  [1.144],
  [1.169],
  [1.057080],
  [1948],
  [1.056],
  [1.107],
  [0.965],
  [1.055012],
  [1949],
  [1.038],
  [1.321],
  [1.133],
  [1.187925],
  [1950],
  [1.089],
  [1.305],
  [1.732],
  [1.317130],
  [1951],
  [1.090],
  [1.195],
  [1.021],
  [1.240164],
  [1952],
  [1.083],
  [1.390],
  [1.131],
  [1.183675],
  [1953],
  [1.035],
  [0.928],
  [1.006],
  [0.990108],
  [1954],
  [1.176],
  [1.715],
  [1.908],
  [1.526236],
  table.hline(),
)

= 思路

```


In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [ ]:
# 导入数据
N = 12
NX = 3
T = 0.15

# 股票收益率
Ra = (
  np.array(
    [
      1.258997,
      1.197526,
      1.364361,
      0.919287,
      1.057080,
      1.055012,
      1.187925,
      1.317130,
      1.240164,
      1.183675,
      0.990108,
      1.526236,
    ]
  )
  - 1
)

R1 = np.array([1.300, 1.103, 1.216, 0.954, 0.929, 1.056, 1.038, 1.089, 1.090, 1.083, 1.035, 1.176]) - 1
R2 = np.array([1.225, 1.290, 1.216, 0.728, 1.144, 1.107, 1.321, 1.305, 1.195, 1.390, 0.928, 1.715]) - 1
R3 = np.array([1.149, 1.260, 1.419, 0.922, 1.169, 0.965, 1.133, 1.732, 1.021, 1.131, 1.006, 1.908]) - 1

In [ ]:
ER1 = np.mean(R1)
ER2 = np.mean(R2)
ER3 = np.mean(R3)

DR1 = np.std(R1)
DR2 = np.std(R2)
DR3 = np.std(R3)

Cov = np.cov(np.array([R1, R2, R3]))

In [ ]:
def ER(X):
  if X.size != 3:
    raise ValueError('X.size must be 3')

  return np.dot(X, np.array([ER1, ER2, ER3]))


def DR(X):
  if X.size != 3:
    raise ValueError('X.size must be 3')

  return np.sqrt(np.dot(np.dot(X, Cov), X))


nlc1 = opt.NonlinearConstraint(ER, T, np.inf)

A2 = np.ones(3)
b2 = 1
nlc2 = opt.NonlinearConstraint(lambda X: np.dot(A2, X), 1, 1)

result = opt.minimize(DR, np.array([1 / 3, 1 / 3, 1 / 3]), constraints=[nlc1, nlc2])

_1X = result.x
print('X:', _1X)
print('ER:', ER(_1X))
print('DR:', result.fun)

A，B，C 股依次占比 0.53008494 0.35646096 0.1134541（%）


In [ ]:
lb = 0.1
ub = ER3
step = 0.001
E = np.arange(lb, ub, step)

nlc4 = opt.NonlinearConstraint(lambda X: X[0], 0, 1)
nlc5 = opt.NonlinearConstraint(lambda X: X[1], 0, 1)
nlc6 = opt.NonlinearConstraint(lambda X: X[2], 0, 1)

for i in range(E.size):
  nlc3 = opt.NonlinearConstraint(ER, E[i], E[i])
  result = opt.minimize(DR, np.array([1 / 3, 1 / 3, 1 / 3]), constraints=[nlc2, nlc3, nlc4, nlc5, nlc6])
  plt.scatter(E[i], result.fun, c='r')
  pass

In [ ]:
ER4 = 0.05

lb_n = ER4
ub_n = ER3
step = 0.001
E_n = np.arange(lb_n, ub_n, step)


def ER_n(X):
  if X.size != 4:
    raise ValueError('X.size must be 4')

  return np.dot(X, np.array([ER1, ER2, ER3, ER4]))


def DR_n(X):
  if X.size != 4:
    raise ValueError('X.size must be 4')

  _X = X[0:3]

  return np.sqrt(np.dot(np.dot(_X, Cov), _X))


A3 = np.ones(4)
nlc7 = opt.NonlinearConstraint(lambda X: np.dot(A3, X), 1, 1)
nlc8 = opt.NonlinearConstraint(lambda X: X[3], 0, 1)


for i in range(E_n.size):
  nlc9 = opt.NonlinearConstraint(ER_n, E_n[i], E_n[i])
  result = opt.minimize(DR_n, np.array([1 / 4, 1 / 4, 1 / 4, 1 / 4]), constraints=[nlc4, nlc5, nlc6, nlc7, nlc8, nlc9])
  plt.scatter(E_n[i], result.fun, c='b')
  pass

nlc9f = opt.NonlinearConstraint(ER_n, T, T)
result = opt.minimize(DR_n, np.array([1 / 4, 1 / 4, 1 / 4, 1 / 4]), constraints=[nlc4, nlc5, nlc6, nlc7, nlc8, nlc9f])

_2X = result.x
print('X:', _2X)
print('DR:', result.fun)

Target = 15% 时

A，B，C 和国库券依次占比 0.08660361 0.42839287 0.1435753 0.34142821（%）


In [ ]:
c0 = np.array([0.5, 0.35, 0.15])


def common(M):
  if M.size != 6:
    raise ValueError('M.size must be 6')

  _Y = M[0:3]  # in
  _Z = M[3:6]  # out

  _X = [c0[i] + _Y[i] - _Z[i] for i in range(3)]

  return _X


A4 = np.array([np.ones(3), np.ones(3) * 0.01, np.ones(3) * 0.01])
nlc10 = opt.NonlinearConstraint(
  lambda M: np.dot(A4[0], common(M)) + np.dot(A4[1], M[0:3]) + np.dot(A4[2], M[3:6]), 1, 1
)


def ER_m(M):
  _X = common(M)

  return np.dot(_X, np.array([ER1, ER2, ER3]))


def DR_m(M):
  _X = common(M)

  return np.sqrt(np.dot(np.dot(_X, Cov), _X))


nlc11 = opt.NonlinearConstraint(ER_m, T, T)

result = opt.minimize(DR_m, np.zeros(6), constraints=[nlc10, nlc11])

_3X = common(result.x)
print('X:', _3X)
print('DR:', result.fun)

Target = 15% 时

A，B，C 股依次买入 1.348e-02 1.987e-03 -2.019e-02（%），卖出 -1.666e-02 -5.173e-03 1.701e-02（%）
